In [ ]:
import os
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%run util.ipynb
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 200)

In [ ]:
s= "AMAZON-DOC_PkwiseSynSearch_theta:0.9,kmax:1_3162277_107836_3_1.0"
s_list = [s]
l = list(map(lambda x:('_'.join(x[0:3]), (int(x[3]), int(x[4]), int(x[5]), float(x[6]), -float(x[2].split(',')[0].split(':')[1]))), map(lambda x:tuple(x.split('_')), s_list)))
l

In [ ]:
s = "timeout -k 1 14400 java -Xmx8g -cp target/substring-syn-0.0.1-SNAPSHOT.jar snu.kdd.substring_syn.App -data WIKI-DOC -nt 10000 -nr 107836 -ql 1 -lr 1.0 -alg PkwiseSynSearch -param theta:0.6,kmax:1 && bash ./upload.sh || (touch MSG && scp -q MSG cherry-manage:/home/hadoop/ghsong/substring_syn/failed/WIKI-DOC_PkwiseSynSearch_theta:0.6,kmax:1_10000_107836_1_1.0 && rm MSG)"
tokens = s.split('&&')[0].split(' ')
d = {}
for i in range(len(tokens)):
    if tokens[i].startswith('-'): d[tokens[i][1:]] = tokens[i+1]
list(map(lambda x:x.split(':')[1], filter(lambda x:x.startswith('theta'), d['param'].split(','))))[0]

In [ ]:
def invalidate_old_expitems():
    query = """
        SELECT id, datetime(exp_date,'localtime') as date, dataset_id as did, algorithm_id as aid, parameter as param
        FROM projectmanager_expitem
        WHERE dataset_id>=429 AND invalid=0
        """
    d = collections.defaultdict(set)
    for row in execute_query(query):
    #     print(row['id'], row['date'], row['did'], row['aid'], str2dict(row['param']))
        key = (row['did'], row['aid'], tuple(sorted(str2dict(row['param']).items())))
        d[key].add((row['date'], row['id']))
    for key in d:
        if len(d[key]) < 2: continue
        vals = list(sorted(d[key], key=lambda x:x[0]))
        for date, expid in vals[:-1]:
            query = """
            UPDATE projectmanager_expitem
            SET invalid=1
            WHERE id=%d
            """
            execute_query(query%expid)

In [ ]:
invalidate_old_expitems()

In [ ]:
def remove_all_expitem(aid):
    query = """
        DELETE FROM projectmanager_expitem
        WHERE algorithm_id=%s
        """%aid
    execute_query(query)

In [ ]:
def display_using_df(query):
    df = pd.DataFrame(list(map(dict, execute_query(query))))
    print(df)
    return df

In [ ]:
query = """
    SELECT *
    FROM projectmanager_expitem
    WHERE algorithm_id=56 and dataset_id=890
"""
display_using_df(query)

In [ ]:
query = """
    UPDATE projectmanager_expitem
    SET invalid=1
    WHERE dataset_id=911
"""
display_using_df(query)

In [ ]:
import re
s = "abcdcd"
m = re.search('b(cd)?', s)
print(m)

In [ ]:
# query = """
#     SELECT *
#     FROM projectmanager_expitem
#     WHERE algorithm_id=47
#         AND parameter LIKE '%\"bLF\": \"false\"%'
#         AND parameter LIKE '%\"bPF\": \"false\"%'
#         AND parameter LIKE '%\"index_impl\": \"None\"%'
# """
# rows = list(execute_query(query))
# for row in rows:
#     new_result = row['result'].replace("\"Alg_Version\": \"6.30\"", "\"Alg_Version\": \"6.32\"")
#     query = f"""
#         INSERT INTO projectmanager_expitem(project_id, server_id, algorithm_id, dataset_id, exp_date, upload_date, failed, invalid, parameter, result)
#         VALUES({row['project_id']}, {row['server_id']}, {56}, {row['dataset_id']}, '{'2020-05-06 15:00:00'}', '{'2020-05-06 15:00:00'}', 0, 0, '{row['parameter']}', '{new_result}')
#     """
#     execute_query(query)